In [63]:
# packages
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib tk

In [65]:
# function to determine if distance between two points is acceptable
def dist(new_pt, pts, min_dist):
    for pt in pts:
        dist = np.sqrt(np.sum(np.square(new_pt - pt)))
        if dist < min_dist:
            return False
    return True

In [67]:
# function to determind closest point in array
def closest_pt(new_pt, pts):
    dist_lst = []
    for pt in pts:
        dist = np.sqrt(np.sum(np.square(new_pt - pt)))
        dist_lst.append(dist)
    return pts[dist_lst.index(min(dist_lst))]

In [69]:
# function to generate random points that are not within a minimum distance of each other
def rand_pts(N, min_dist, res):
    pts = []
    scope = np.arange(0,res,1)
    while len(pts) < N:
        new_pt = np.random.choice(scope, 2)
        if dist(new_pt, pts, min_dist):
            pts.append(new_pt)
    return pts

In [71]:
# function to generate random star sizes
def rand_size(N):
    size = []
    scope = np.arange(1,5,1)
    while len(size) < N:
        new_size = np.random.choice(scope)
        size.append(new_size)
    return size

In [73]:
# function to generate random star blur
def rand_blur(N):
    blur = []
    scope = np.arange(0,1,1)
    while len(blur) < N:
        new_blur = np.random.choice(scope)
        blur.append(new_blur)
    return blur

In [75]:
# 1D random walk function to simulate atmosphere fluctuations
def rand_walk_1d(N, step):
    y = 0
    lst = []
    while len(lst) < N:
        y += np.random.randint(-step, step + 1)
        lst.append(y)
    return np.array(lst)

In [77]:
# 2D random walk function to simulate atmosphere fluctuations
def rand_walk_2d(N, step):
    x = 0
    y = 0
    lst = []
    lst = []
    while len(lst) < N:
        y += np.random.randint(-step, step + 1)
        x += np.random.randint(-step, step + 1)
        lst.append([x,y])
    return np.array(lst)

In [79]:
# define function to generate image of stars
def stars(res, points):
    # create black background
    background = np.zeros((res,res,3), np.uint8)

    # make background grey-er [25, 25, 25]
    background[background < 1] = 25

    # create the mask of same shape
    mask = np.zeros((res,res), np.uint8)

    # generate random size of star
    star_size = rand_size(res)

    # draw circles onto the mask
    for i in range(len(points)):
        mask = cv.circle(mask, points[i], star_size[i], (255, 255, 255), -1, cv.LINE_AA)
    
    # apply Gaussian blurp
    mask = cv.GaussianBlur(mask, (9,9), 2)
    
    # for blending purposes create 3-channel mask
    alpha = cv.cvtColor(mask, cv.COLOR_GRAY2BGR)/255.0

    # create a coloured image
    coloured = np.zeros(background.shape, np.uint8)
    coloured[:,:,:] = [255, 255, 255]

    # perform alpha-blending and convert to integer
    blended = cv.convertScaleAbs(background*(1-alpha) + coloured*alpha)
    
    # return the image
    return blended

In [81]:
# define function to shift values in image array
def val_shift(base_img, shift):
    # create copy of image
    new_img = base_img

    # shift values in image array
    new_img = np.array(new_img,dtype=np.int16)
    new_img = [x - shift for x in new_img]

    # cap maximum and minimum values
    new_img = np.where(np.array(new_img)<0, 0, np.array(new_img))
    new_img = np.where(np.array(new_img)>254, 255, np.array(new_img))

    return new_img

In [83]:
# define function to jiggle images
def img_jiggle(base_img, shift, crop):
    # apply jiggle crop
    if -crop < shift[0] < crop:
        crop_img = base_img[(crop+shift[0]):(600-crop+shift[0]), 0:600]
    else:
        if shift[0] >= crop:
            crop_img = base_img[(2*crop):600,0:600]
        if shift[0] <= -crop:
            crop_img = base_img[0:(600-2*crop),0:600]

    y = len(crop_img)
    
    if -crop < shift[1] < crop:
        crop_img = crop_img[0:y, (crop+shift[1]):(600-crop+shift[1])]
    else:
        if shift[1] >= crop:
            crop_img = crop_img[0:y, (2*crop):600]
        if shift[1] <= -crop:
            crop_img = crop_img[0:y, 0:(600-2*crop)]

    return crop_img

In [85]:
# define function to simulate transit event
def transit_fun(x, r):
    if 0 <= x <= 2*r:
        return (np.arccos(1-x/r)*r**2 - r**2*np.sqrt(1-(1-x/r)**2)*(1-x/r))/(np.pi*r**2)
    
    if 2*r < x < 6*r:
        return 1
        
    if 6*r <= x <= 8*r:
        x = x - 6*r
        return 1 - (np.arccos(1-x/r)*r**2 - r**2*np.sqrt(1-(1-x/r)**2)*(1-x/r))/(np.pi*r**2)

def full_transit_fun(length):
    t_pts_transit = np.linspace(0,2*round(length/3),2*round(length/3))
    x_pts = []
    for i in range(0,length-5*round(length/6)):
        x_pts.append(0)
        
    for i in range(0,2*round(length/3)):
        #print(transit_fun(t_pts_transit[i], round(length/3)/4))
        x_pts.append(0.01*transit_fun(t_pts_transit[i], round(length/3)/4))
        
    for i in range(0,length-(length-5*round(length/6) + 2*round(length/3))):
        x_pts.append(0)
    return x_pts

In [113]:
# empty folder
for f in os.listdir('star-images/'):
    if not f.endswith(".jpg"):
        continue
    os.remove(os.path.join('star-images/', f))


# generate atmosphere fluctuations for N = 100 images
atm = rand_walk_1d(100, 3.5)

# generate imprecise telescope tracking movement
jiggle = rand_walk_2d(100, 6)

# generate stars with minimum distance
points = rand_pts(50, 40, 600)

# generate original image of stars
base_img = stars(600, points)


# select transit star
plt.imshow(base_img)
coords = plt.ginput(1)[0]
plt.close() 

# find star clicked
star_coords = closest_pt(coords, points)

# add dark grey circular mask over star location 
covered_star = cv.circle(base_img.copy(), star_coords, 10, (25,25,25), -1, cv.LINE_AA)

# create and save a version of the image with red circle to locate the star
circled_star = cv.circle(base_img.copy(), star_coords, 15, (0,0,255), 1, cv.LINE_AA)
circled_star = img_jiggle(circled_star, jiggle[0], 65)
cv.imwrite('star-images/reference-image.jpg', circled_star)

# generate transit curve
weight = full_transit_fun(len(atm))


# generate N = 100 images of stars over time
for i in range(0,len(atm)):
    # apply transit brightness reduction to simulate exoplanet transit
    new_img_transit = cv.addWeighted(covered_star, weight[i], base_img, 1-weight[i], 0)
    
    # apply image brightness shift to simulate atmosphere changes
    new_img_shift = val_shift(new_img_transit, atm[i])

    # apply jiggly cropping to simulate imperfect tracking by the telescope
    new_img_jiggle = img_jiggle(new_img_shift, jiggle[i], 65)
    
    # save image in folder
    cv.imwrite('star-images/image-' + str(i) + '.jpg', new_img_jiggle)